In [1]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

from src.features import load_team_opponent
from src.features.preprocess import derived_odds,get_bookmaker, get_odd_pred, split_date,add_last_3_scores_column
import numpy as np

# Lade Daten ein

## Team Opponent

In [10]:
from src.utils.utils import load_from_pickle

In [77]:
df = load_from_pickle(source_path= CONFIG.DATA_FOLDER_RAW+'raw_database.sav')


In [78]:
df['Date'] = pd.to_datetime(df['Date'])

In [79]:
df = df[~df.B365H.isna() & ~df.IWH.isna()]

In [152]:
X = df[["HomeTeam","AwayTeam","Div","B365H","B365D","B365A","BWH","BWD","BWA","IWH","IWD","IWA"]]

In [180]:
y = df[["B365H","B365D","B365A","FTR"]]
y.columns =["H","D","A","R"]

In [181]:
import pandas as pd

def make_y(y_row):
    r = y_row["R"]
    y_row[r] = y_row[r]-1
    results = {'H','D','A'}
    y_row[list(results - {r})] = 0
    return y_row


# apply the make_y function to every row of the DataFrame
y= y.apply(lambda x: make_y(x), axis=1)

y = y.drop("R",axis=1)
# print the updated DataFrame
print(y)


       H     D     A
0    0.0  2.50  0.00
1    0.0  2.20  0.00
2    0.0  2.75  0.00
3    0.0  2.20  0.00
4    1.1  0.00  0.00
..   ...   ...   ...
235  0.0  2.60  0.00
236  1.2  0.00  0.00
237  0.0  2.60  0.00
238  0.0  0.00  1.55
239  0.0  0.00  3.33

[6996 rows x 3 columns]


In [186]:
import numpy as np

def set_max_row(arr):
    """
    Sets the maximum value in each row of a numpy array to 1 and all other values to 0.
    """
    # Find indices of maximum values in each row
    max_indices = np.argmax(arr, axis=1)

    # Set all values to 0 except for the maximum value in each row
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, :] = 0
        arr[i, max_index] = 1

    return arr

wins = y_test.sum(axis=1)
output = (set_max_row(y_test.values)*set_max_row(preds)).sum(axis= 1)*wins
print(wins)

269    1.0
262    1.0
213    1.0
265    1.0
143    1.0
      ... 
204    1.0
179    1.0
49     1.0
31     1.0
12     1.0
Length: 1749, dtype: float64


In [191]:
y_test

,H,D,A
269,1.0,0.0,0.0
262,0.0,1.0,0.0
213,1.0,0.0,0.0
265,1.0,0.0,0.0
143,1.0,0.0,0.0
...,...,...,...
204,1.0,0.0,0.0
179,0.0,0.0,1.0
49,0.0,1.0,0.0
31,1.0,0.0,0.0


In [190]:
set_max_row(model.predict(X_test.values))

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [192]:
y

,H,D,A
0,0.0,2.50,0.00
1,0.0,2.20,0.00
2,0.0,2.75,0.00
3,0.0,2.20,0.00
4,1.1,0.00,0.00
...,...,...,...
235,0.0,2.60,0.00
236,1.2,0.00,0.00
237,0.0,2.60,0.00
238,0.0,0.00,1.55


In [172]:
(output.sum()-sum(output == 0))/len(output)

0.06450543167524296

In [149]:
pd.DataFrame().sum(axis=1)*y.sum(axis=1)

0       0.0
0       0.0
0       0.0
0       0.0
0       0.0
       ... 
6991    NaN
6992    NaN
6993    NaN
6994    NaN
6995    NaN
Length: 13686, dtype: float64

In [182]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# Load dataset

# Convert categorical variables to numerical labels
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

# Define TabNet model
model = TabNetRegressor(n_d=32, n_a=32, n_steps=5, gamma=1.3, lambda_sparse=0, optimizer_fn=torch.optim.Adam)

# Train model
model.fit(X_train.values, y_train.values, max_epochs=100, patience=10, batch_size=1024)

# Make predictions
preds = model.predict(X_test.values)


C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\pytorch_tabnet\abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4.63193 |  0:00:00s
epoch 1  | loss: 2.48165 |  0:00:01s
epoch 2  | loss: 1.96015 |  0:00:02s
epoch 3  | loss: 1.8285  |  0:00:02s
epoch 4  | loss: 1.65038 |  0:00:03s
epoch 5  | loss: 1.68286 |  0:00:04s
epoch 6  | loss: 1.59782 |  0:00:05s
epoch 7  | loss: 1.52427 |  0:00:06s
epoch 8  | loss: 1.50131 |  0:00:06s
epoch 9  | loss: 1.50599 |  0:00:07s
epoch 10 | loss: 1.49307 |  0:00:08s
epoch 11 | loss: 1.46663 |  0:00:09s
epoch 12 | loss: 1.46561 |  0:00:10s
epoch 13 | loss: 1.42813 |  0:00:11s
epoch 14 | loss: 1.44125 |  0:00:12s
epoch 15 | loss: 1.44584 |  0:00:13s
epoch 16 | loss: 1.43709 |  0:00:14s
epoch 17 | loss: 1.43124 |  0:00:14s
epoch 18 | loss: 1.42012 |  0:00:15s
epoch 19 | loss: 1.43329 |  0:00:16s
epoch 20 | loss: 1.44892 |  0:00:17s
epoch 21 | loss: 1.44109 |  0:00:18s
epoch 22 | loss: 1.4317  |  0:00:19s
epoch 23 | loss: 1.41864 |  0:00:20s
epoch 24 | loss: 1.42015 |  0:00:21s
epoch 25 | loss: 1.42635 |  0:00:22s
epoch 26 | loss: 1.42874 |  0:00:23s
e

In [ ]:
class Precision(Metric):
    def __init__(self):
        self._name = "precision"
        self._maximize = True

    def __call__(self, y_true, y_pred):
        max(y_true)
        return precision

In [183]:
y_test

,H,D,A
269,0.67,0.0,0.0
262,0.00,6.0,0.0
213,0.36,0.0,0.0
265,0.75,0.0,0.0
143,0.55,0.0,0.0
...,...,...,...
204,0.72,0.0,0.0
179,0.00,0.0,3.0
49,0.00,2.5,0.0
31,1.30,0.0,0.0


In [184]:
pd.Series(model.feature_importances_,index=X.columns)

HomeTeam    0.072786
AwayTeam    0.129462
Div         0.154044
B365H       0.041293
B365D       0.016655
B365A       0.062869
BWH         0.145439
BWD         0.101780
BWA         0.130276
IWH         0.020557
IWD         0.050760
IWA         0.074078
dtype: float64

In [118]:
import pandas as pd
import numpy as np

def set_max_row(df):
    """
    Sets everything but the maximum value in each row of a pandas dataframe to 0.
    """
    # Convert dataframe to numpy array
    arr = df.values

    # Find indices of maximum values in each row
    max_indices = np.argmax(arr, axis=1)

    # Set all values before the maximum to 0
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, :max_index] = 0

    # Set all values after the maximum to 0
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, max_index+1:] = 0

    # Convert back to pandas dataframe
    result_df = pd.DataFrame(arr, columns=df.columns, index=df.index)

    return result_df


In [4]:
sum(df.B365_Opponent.isna())

4

## Sortiere den Datensatz nach Datum, für die Durchschnittsfelder

In [5]:
df = split_date(df)
df = df.sort_values(['year','month','day'],ascending=True)
df = df.reset_index()
df.Date = pd.to_datetime(df.Date)

In [5]:
df["diff_FTG_against_Opponent"] = df["FTG_Team"]-df["FTG_Opponent"]
df["diff_ShotsOnTarget_against_Opponent"] = df["ShotsOnTarget_Team"]-df["ShotsOnTarget_Opponent"]

# Erstelle Zielvariable

In [6]:
df['Target'] = df.FTG_Team>df.FTG_Opponent
df['Target'].value_counts()

False    8762
True     5264
Name: Target, dtype: int64

# Features entwicklen

In [7]:
standad_attributes = ['Target','atHome','Date','Div']
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)
bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
bet_on_opponent = get_bookmaker(bm='Opponent',df=df[bookmaker_attributes])
bet_on_draw = get_bookmaker(bm='Draw',df=df[bookmaker_attributes])


In [8]:
df_derived_b365 = get_odd_pred(bet="B365",df=df)
df_derived_iw = get_odd_pred(bet="IW",df=df)
df_derived_bw = get_odd_pred(bet="BW",df=df)


In [9]:
df_derived_team = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
df_derived_opponent = derived_odds(sight = 'Opponent',df=df, odds=bet_on_opponent)
df_derived_draw = derived_odds(sight = 'Draw',df=df, odds=bet_on_draw)


In [10]:
# df = add_last_3_scores_column(df = df, score_column="diff_FTG_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="diff_ShotsOnTarget_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Team",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Opponent",anz_games=4)
# df = df.dropna()
# rolling_mean_attributes = [c for c in  df.columns if 'last' in c]

# Modelldatensatz erstellen

In [11]:
model_data = pd.concat([df[standad_attributes],
                        #df[rolling_mean_attributes],
                        df[bookmaker_attributes],
                        df[["Team","Opponent","month","year"]],
                        df_derived_team,
                        df_derived_opponent,
                        df_derived_draw,
                        df_derived_b365,
                        df_derived_iw,
                        df_derived_bw],axis=1)

In [12]:
model_data.replace([np.inf, -np.inf], np.nan, inplace=True)

#model_data = model_data.dropna()

In [13]:
df.IW_Team.isna().sum()

30

## Encoding Teams

In [14]:
from sklearn import preprocessing
le_div = preprocessing.LabelEncoder()
le_team = preprocessing.LabelEncoder()


In [15]:
le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)
model_data.Opponent = le_team.transform(model_data.Opponent)

In [16]:
le_div.fit(list(set(model_data.Div)))
model_data.Div = le_div.transform(model_data.Div)


Aufteilen in Training, Test und Validierung

In [17]:
model_data.Date = pd.to_datetime(model_data.Date)

In [18]:
import datetime
datum_1_1_2020 = datetime.datetime(2021, 6, 1)
test_data = model_data[model_data.Date>datum_1_1_2020]
model_data = model_data[model_data.Date<datum_1_1_2020]


In [19]:
datum_1_1_2020 = datetime.datetime(2019, 6, 1)
model_data_valid = model_data[model_data.Date>datum_1_1_2020]
model_data_train = model_data[model_data.Date<datum_1_1_2020]

In [20]:
len(model_data_train)

9472

In [21]:
model_data_train.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Train/model_data.csv')
model_data_valid.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Valid/model_data.csv')
test_data.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Test/model_data.csv')
